In [1]:
import numpy as np
import pandas as pd
import ipaddress
import itertools
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import socket
import requests
from ip2geotools.databases.noncommercial import DbIpCity
from geopy.distance import distance

CSVs from: https://dev.maxmind.com/geoip/geolite2-free-geolocation-data?lang=en


Geoname_id from: https://www.geonames.org/

Downloads: https://www.geonames.org/export/

Data dictionary: https://dev.maxmind.com/geoip/docs/databases/city-and-country?lang=en

[+] You need to review IPv4 and IPv6 formatting and develop code to parse.


City locations contains all of country locations.


In [2]:
ipv4_city_blocks = pd.read_csv('GeoLite2-City-Blocks-IPv4.csv')

In [3]:
ipv4_city_blocks.head()

,network,geoname_id,registered_country_geoname_id,represented_country_geoname_id,is_anonymous_proxy,is_satellite_provider,postal_code,latitude,longitude,accuracy_radius
0,1.0.0.0/24,2158177.0,2077456.0,NaN,0,0,3135,-37.8333,145.2375,1000.0
1,1.0.1.0/24,1814991.0,1814991.0,NaN,0,0,NaN,34.7732,113.7220,1000.0
2,1.0.2.0/23,1814991.0,1814991.0,NaN,0,0,NaN,34.7732,113.7220,1000.0
3,1.0.4.0/23,2158177.0,2077456.0,NaN,0,0,3109,-37.7789,145.1600,1000.0
4,1.0.6.0/23,2147714.0,2077456.0,NaN,0,0,2000,-33.8715,151.2006,500.0


In [4]:
city_locs = pd.read_csv('GeoLite2-City-Locations-en.csv')

In [5]:
ipv4_city_blocks = ipv4_city_blocks.merge(city_locs, on='geoname_id', how='left')

In [6]:
cols_to_keep = ['network', 'city_name','subdivision_1_name', 'country_name', 'latitude', 'longitude']

In [7]:
ipv4_df = ipv4_city_blocks[cols_to_keep].copy()

In [8]:
ipv4_df.rename({'city_name':'city', 'subdivision_1_name':'region', 'country_name':'country'}, axis=1, inplace=True)

In [9]:
ipv4_df.isna().sum()

network           0
city         372804
region       364538
country        1399
latitude        145
longitude       145
dtype: int64

In [10]:
ipv4_df.dropna(subset = 'latitude', inplace = True)

In [11]:
#Create a better csv?
#ipv4_df.to_csv("ipv4_networks.csv", index=False)

### Inital searches took too long. Creating smaller bins for quicker searching:


In [12]:
df = pd.read_csv('curriculum_logs.csv')

In [14]:
ip_addys = df.ip.unique()

In [15]:
ip_addys

array(['97.105.19.61', '71.137.146.71', '209.205.123.242', ...,
       '134.250.1.82', '174.197.2.198', '107.77.169.13'], dtype=object)

In [16]:
addy_list = []
for item in ip_addys:
    addy_list.append(ipaddress.ip_address(item))

In [17]:
type(addy_list[4])

ipaddress.IPv4Address

In [18]:
network_list = []
for thing in ipv4_df.network:
    network_list.append(ipaddress.ip_network(thing))

In [19]:
type(network_list[7])

ipaddress.IPv4Network

In [20]:
network_list.sort()

In [21]:
class_a_ip_addy = [ipaddress.ip_address("0.0.0.0") , ipaddress.ip_address("127.255.255.255")]
class_b_ip_addy = [ipaddress.ip_address("128.0.0.0") , ipaddress.ip_address("191.255.255.255")]
class_c_ip_addy = [ipaddress.ip_address("192.0.0.0") , ipaddress.ip_address("223.255.255.255")]
class_d_ip_addy = [ipaddress.ip_address("224.0.0.0") , ipaddress.ip_address("239.255.255.255")]
class_e_ip_addy = [ipaddress.ip_address("240.0.0.0") , ipaddress.ip_address("255.255.255.255")]

In [22]:
class_a0_ip_net = ipaddress.ip_network("16.0.0.0/32")
class_a1_ip_net = ipaddress.ip_network("32.0.0.0/32")
class_a2_ip_net = ipaddress.ip_network("48.0.0.0/32")
class_a3_ip_net = ipaddress.ip_network("64.0.0.0/32")
class_a4_ip_net = ipaddress.ip_network("80.0.0.0/32")
class_a5_ip_net = ipaddress.ip_network("96.0.0.0/32")
class_a6_ip_net = ipaddress.ip_network("112.0.0.0/32")
class_b0_ip_net = ipaddress.ip_network("128.0.0.0/32")
class_b1_ip_net = ipaddress.ip_network("144.0.0.0/32")
class_b2_ip_net = ipaddress.ip_network("160.0.0.0/32")
class_b3_ip_net = ipaddress.ip_network("176.0.0.0/32")
class_c0_ip_net = ipaddress.ip_network("192.0.0.0/32")
class_c1_ip_net = ipaddress.ip_network("208.0.0.0/32")
class_d0_ip_net = ipaddress.ip_network("224.0.0.0/32")
class_e0_ip_net = ipaddress.ip_network("240.0.0.0/32")

In [23]:
class_a0_networks = []
class_a1_networks = []
class_a2_networks = []
class_a3_networks = []
class_a4_networks = []
class_a5_networks = []
class_a6_networks = []
class_b0_networks = []
class_b1_networks = []
class_b2_networks = []
class_b3_networks = []
class_c0_networks = []
class_c1_networks = []
class_d0_networks = []
class_e0_networks = []

In [24]:
for network in ipv4_df.network:
    
    network = ipaddress.ip_network(network)
    
    if network <= class_a0_ip_net:
        class_a0_networks.append(network)
    elif network <= class_a1_ip_net:
        class_a1_networks.append(network)
    elif network <= class_a2_ip_net:
        class_a2_networks.append(network)
    elif network <= class_a3_ip_net:
        class_a3_networks.append(network)
    elif network <= class_a4_ip_net:
        class_a1_networks.append(network)
    elif network <= class_a5_ip_net:
        class_a5_networks.append(network)
    elif network <= class_a6_ip_net:
        class_a6_networks.append(network)
    elif network <= class_b0_ip_net:
        class_b0_networks.append(network)
    elif network <= class_b1_ip_net:
        class_b1_networks.append(network)
    elif network <= class_b2_ip_net:
        class_b2_networks.append(network)
    elif network <= class_b3_ip_net:
        class_b3_networks.append(network)
    elif network <= class_c0_ip_net:
        class_c0_networks.append(network)
    elif network <= class_c1_ip_net:
        class_c1_networks.append(network)
    elif network <= class_d0_ip_net:
        class_d0_networks.append(network)
    else:
        class_e0_networks.append(network)

In [25]:
print(f'Class A0 length: {len(class_a0_networks)}')
print(f'Class A1 length: {len(class_a1_networks)}')
print(f'Class A2 length: {len(class_a2_networks)}')
print(f'Class A3 length: {len(class_a3_networks)}')
print(f'Class A4 length: {len(class_a4_networks)}')
print(f'Class A5 length: {len(class_a5_networks)}')
print(f'Class A6 length: {len(class_a6_networks)}')
print(f'Class B0 length: {len(class_b0_networks)}')
print(f'Class B1 length: {len(class_b1_networks)}')
print(f'Class B2 length: {len(class_b2_networks)}')
print(f'Class B3 length: {len(class_b3_networks)}')
print(f'Class C0 length: {len(class_c0_networks)}')
print(f'Class C1 length: {len(class_c1_networks)}')
print(f'Class D0 length: {len(class_d0_networks)}')
print(f'Class E0 length: {len(class_e0_networks)}')

Class A0 length: 93830
Class A1 length: 890429
Class A2 length: 206957
Class A3 length: 184402
Class A4 length: 0
Class A5 length: 431566
Class A6 length: 414770
Class B0 length: 130724
Class B1 length: 122148
Class B2 length: 140293
Class B3 length: 189817
Class C0 length: 311872
Class C1 length: 260636
Class D0 length: 219573
Class E0 length: 0


In [26]:
class_a0_address = []
class_a1_address = []
class_a2_address = []
class_a3_address = []
class_a4_address = []
class_a5_address = []
class_a6_address = []
class_b0_address = []
class_b1_address = []
class_b2_address = []
class_b3_address = []
class_c0_address = []
class_c1_address = []
class_d0_address = []
class_e0_address = []

In [29]:
for address in addy_list:
    
    if address < ipaddress.ip_address('17.0.0.0'):
        class_a0_address.append(address)
    elif address < ipaddress.ip_address('33.0.0.0'):
        class_a1_address.append(address)
    elif address < ipaddress.ip_address('49.0.0.0'):
        class_a2_address.append(address)
    elif address < ipaddress.ip_address('65.0.0.0'):
        class_a3_address.append(address)
    elif address < ipaddress.ip_address('81.0.0.0'):
        class_a1_address.append(address)
    elif address < ipaddress.ip_address('97.0.0.0'):
        class_a5_address.append(address)
    elif address < ipaddress.ip_address('113.0.0.0'):
        class_a6_address.append(address)
    elif address < ipaddress.ip_address('129.0.0.0'):
        class_b0_address.append(address)
    elif address < ipaddress.ip_address('145.0.0.0'):
        class_b1_address.append(address)
    elif address < ipaddress.ip_address('161.0.0.0'):
        class_b2_address.append(address)
    elif address < ipaddress.ip_address('177.0.0.0'):
        class_b3_address.append(address)
    elif address < ipaddress.ip_address('193.0.0.0'):
        class_c0_address.append(address)
    elif address < ipaddress.ip_address('209.0.0.0'):
        class_c1_address.append(address)
    elif address < ipaddress.ip_address('225.0.0.0'):
        class_d0_address.append(address)
    else:
        class_e0_address.append(network)

In [30]:
print(f'Class A0 length: {len(class_a0_address)}')
print(f'Class A1 length: {len(class_a1_address)}')
print(f'Class A2 length: {len(class_a2_address)}')
print(f'Class A3 length: {len(class_a3_address)}')
print(f'Class A4 length: {len(class_a4_address)}')
print(f'Class A5 length: {len(class_a5_address)}')
print(f'Class A6 length: {len(class_a6_address)}')
print(f'Class B0 length: {len(class_b0_address)}')
print(f'Class B1 length: {len(class_b1_address)}')
print(f'Class B2 length: {len(class_b2_address)}')
print(f'Class B3 length: {len(class_b3_address)}')
print(f'Class C0 length: {len(class_c0_address)}')
print(f'Class C1 length: {len(class_c1_address)}')
print(f'Class D0 length: {len(class_d0_address)}')
print(f'Class E0 length: {len(class_e0_address)}')

Class A0 length: 77
Class A1 length: 1535
Class A2 length: 156
Class A3 length: 105
Class A4 length: 0
Class A5 length: 194
Class A6 length: 1318
Class B0 length: 2
Class B1 length: 190
Class B2 length: 44
Class B3 length: 1544
Class C0 length: 126
Class C1 length: 164
Class D0 length: 76
Class E0 length: 0


### Create internal database
- How to handle ip address versus network address?
- Check subnet on network to see if smaller network exists for more precise location.

### Create function to check if ip is in internal database
- Add function to append date, check if ip is >1 month old.

### Create function to query external source < 900times / day
- Top 500 and bottom 400 values counts.
- Priority would be for new IPs, then ones >1 month old.

### Build a function for comparison:


In [15]:
addy_file = "curriculum_logs.csv"

def addy_giver(addy_file):
    df = pd.read_csv(addy_file)
    ip_addys = df.ip.unique()
    ip_addys.sort()
    for address in  ip_addys:
        yield ipaddress.ip_address(address)
        

In [16]:
#Try using more generators:
gen_file = "ipv4_networks.csv"

def network_giver(gen_file):
    df = pd.read_csv(gen_file)
    ip_networks = df.network
    for network in  ip_networks:
        yield ipaddress.ip_network(network)
        

In [17]:
address = ipaddress.ip_address('174.197.2.198')
"""
i = 0
for network in network_giver(gen_file):
    if address in network:
        print(f'{address.reverse_pointer} in {network} found.')
        break
    i += 1
    if i % 100_000 == 0:
        print(f'{i} networks searched.')
"""

"\ni = 0\nfor network in network_giver(gen_file):\n    if address in network:\n        print(f'{address.reverse_pointer} in {network} found.')\n        break\n    i += 1\n    if i % 100_000 == 0:\n        print(f'{i} networks searched.')\n"

ChatGPT Code:


gen_file = "ipv4_networks.csv"
addy_file = "curriculum_logs.csv"


In [19]:
def ip_details(ip):
    """
    Uses ip2geotools to get information for ip addresses. Limited to 1000/day for the free tier.
    """
    
    res = DbIpCity.get(ip, api_key="free")
    output =  {"ip":res.ip_address, 
               "city": res.city, 
               "region":res.region,
               "country":res.country,
               "latitude":res.latitude,
               "longitude": res.longitude}
    return output

ip_list = []
i = 0
for item in ip_addys:
ip_list.append(ip_details(item))
i += 1
if i % 10 == 0:
print(f'{i} ip addresses resolved.')


In [ ]:
ip_list[-5:]

def csv_reader(file_path):
with open(file_path, "r") as f:
for row in f:
yield row.strip()


def addy_giver(file_path):
df = pd.read_csv(file_path)
for address in df.ip.unique():
yield ipaddress.ip_address(address)


for network_str in csv_reader(gen_file):
try:
network = ipaddress.ip_network(network_str)
except ValueError:
continue
addrs = set(addy_giver(addy_file))
network_addrs = set(network.hosts())
common_addrs = addrs.intersection(network_addrs)
for dis_addy in common_addrs:
print(f'{dis_addy} is in {network_str}')


In [17]:
"""
gen_file = "ipv4_networks.csv"
addy_file = "curriculum_logs.csv"

def csv_reader(file_path):
    with open(file_path, "r") as f:
        for row in f:
            yield row.strip()

def addy_giver(file_path):
    df = pd.read_csv(file_path)
    for address in df.ip.unique():
        try:
            yield ipaddress.ip_address(address)
        except ValueError:
            print(f"Invalid IP address: {address}")

for network_str in csv_reader(gen_file):
    try:
        network = ipaddress.ip_network(network_str)
    except ValueError:
        print(f"Invalid network address: {network_str}")
        continue
    addrs = set()
    for address in addy_giver(addy_file):
        try:
            addrs.add(address)
        except ValueError:
            print(f"Invalid IP address: {address}")
    network_addrs = set(network.hosts())
    common_addrs = addrs.intersection(network_addrs)
    for dis_addy in common_addrs:
        print(f'{dis_addy} is in {network_str}')
"""

'\ngen_file = "ipv4_networks.csv"\naddy_file = "curriculum_logs.csv"\n\ndef csv_reader(file_path):\n    with open(file_path, "r") as f:\n        for row in f:\n            yield row.strip()\n\ndef addy_giver(file_path):\n    df = pd.read_csv(file_path)\n    for address in df.ip.unique():\n        try:\n            yield ipaddress.ip_address(address)\n        except ValueError:\n            print(f"Invalid IP address: {address}")\n\nfor network_str in csv_reader(gen_file):\n    try:\n        network = ipaddress.ip_network(network_str)\n    except ValueError:\n        print(f"Invalid network address: {network_str}")\n        continue\n    addrs = set()\n    for address in addy_giver(addy_file):\n        try:\n            addrs.add(address)\n        except ValueError:\n            print(f"Invalid IP address: {address}")\n    network_addrs = set(network.hosts())\n    common_addrs = addrs.intersection(network_addrs)\n    for dis_addy in common_addrs:\n        print(f\'{dis_addy} is in {netw

In [18]:
gen_file = "ipv4_networks.csv"
addy_file = "curriculum_logs.csv"

def csv_reader(file_path):
with open(file_path, "r") as f:
next(f) # skip the header row
for row in f:
columns = row.strip().split(",")
try:
yield columns[0] # extract the network address column
except IndexError:
pass


def addy_giver(file_path):
df = pd.read_csv(file_path)
for address in df.ip.unique():
try:
yield ipaddress.ip_address(address)
except ValueError:
print(f"Invalid IP address: {address}")


for network_str in csv_reader(gen_file):
try:
network = ipaddress.ip_network(network_str)
except ValueError:
print(f"Invalid network address: {network_str}")
continue
addrs = set()
for address in addy_giver(addy_file):
try:
addrs.add(address)
except ValueError:
print(f"Invalid IP address: {address}")
network_addrs = set(network.hosts())
common_addrs = addrs.intersection(network_addrs)
for dis_addy in common_addrs:
print(f'{dis_addy} is in {network_str}')



# Define a generator function to read the ipv4_networks.csv file
def read_ipv4_networks():
    with open('ipv4_networks.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Yield an IP network object for each row
            yield ipaddress.ip_network(row['network'])

# Define a generator function to read the curriculum_logs.csv file
def read_curriculum_logs():
    with open('curriculum_logs.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Yield an IP address object for each row
            yield ipaddress.IPv4Address(row['ip'])

# Use set intersection to find the IP addresses in both files
matched_ips = set(read_ipv4_networks()).intersection(set(read_curriculum_logs()))

# Print the matched IP addresses
for ip in matched_ips:
    print(ip)




def load_networks(filename):
    with open(filename) as f:
        reader = csv.reader(f)
        header = next(reader)
        for row in reader:
            network_str = row[0]
            try:
                network = ipaddress.IPv4Network(network_str)
                pass
                #yield network
            except ValueError as e:
                print(f"Error: {e} on line {reader.line_num} with network {network_str}")
                continue

def match_ips_to_networks(networks_filename, ips_filename):
    networks = load_networks(networks_filename)

    with open(ips_filename) as f:
        reader = csv.reader(f)
        header = next(reader)
        for row in reader:
            ip_str = row[5]
            try:
                ip = ipaddress.IPv4Address(ip_str)
            except ValueError as e:
                print(f"Error: {e} on line {reader.line_num} with IP {ip_str}")
                continue
            
            matched = False
            for network in networks:
                if ip in network:
                    matched = True
                    print(f"IP {ip_str} matches network {network}")
                    break
            if not matched:
                pass
                #print(f"IP {ip_str} does not match any network")


In [33]:
gen_file = "ipv4_networks.csv"
addy_file = "curriculum_logs.csv"

In [34]:
#list(load_networks(gen_file))

TypeError: 'NoneType' object is not iterable

In [35]:
#match_ips_to_networks(gen_file, addy_file)

TypeError: 'NoneType' object is not iterable

list_to_join = []

for network in ipv4_df["network"]:
network = ipaddress.ip_network(network)
for item in ip_addys:
item = ipaddress.ip_address(item)

        if item in network:
            ip_dict = {'ip': item,
                       'geoname_id': ipv4_df[ipv4_df["network"] == network]['geoname_id']}
            print('+')
            list_to_join.append(ip_dict)



change to numpy?

list_to_join = []

for network in ipv4_df[:, 1]:
network = ipaddress.ip_network(network)
for item in ip_addys:
item = ipaddress.ip_address(item)

        if item in network:
            ip_dict = {'ip': item,
                       'geoname_id': ipv4_df[ipv4_df["network"] == network]['geoname_id']} <--Unsure how
            print('+')
            list_to_join.append(ip_dict)



list_to_join = [{'ip': ipaddress.ip_address(item),'geoname_id': ipv4_df[ipv4_df["network"] == ipaddress.ip_network(network)]['geoname_id']}
for item in ip_addys['ip'] for network in ipv4_df["network"] if ipaddress.ip_address(item) in ipaddress.ip_network(network)]


list_to_join = [{'ip': ipaddress.ip_address(item),'geoname_id': ipv4_df[ipv4_df["network"] == ipaddress.ip_network(network)]['geoname_id']}
for item in ip_addys['ip'] if not any(ipaddress.ip_address(item) in ipaddress.ip_network(network) for network in ipv4_df["network"])]


In [ ]:
#df_ip = pd.DataFrame(list_to_join)                   

In [ ]:
#df_ip.shape

In [ ]:
#df_ip.to_csv("ip_to_geoname_id.csv", index=False)
